# IMPORTS

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split as tts
from scipy.special import softmax

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def MAPE(y_true, y_pred):
    # computes the MAPE
    return np.mean(np.abs(y_true - y_pred) / y_true)

# Load datasets

In [ ]:
visits = pd.read_parquet('train_visits.parquet.gzip')
ports = pd.read_parquet('ports_prep.parquet.gzip')
vessels = pd.read_parquet('vessels_prep.parquet.gzip')

ports = ports.set_index('port_index')

# Define the dataframe
- Select the instances where a target is present
- Sort values for the split
- Drop irrelevant columns
- Add an identifier

In [ ]:
df = visits[visits['target_travel_duration'].notnull()]
df = df.sort_values('entry_datetime')

df = df.drop(['entry_datetime', 'target_entry_datetime', 
                      'exit_datetime', 'previous_exit_datetime',
                      'target_stay_duration', 'previous_portname', 
                      'previous_entry_datetime', 'prev2_exit_datetime'], 1)


df['identifier'] = df['mmsi'].astype(str) + '_' + df.groupby('mmsi').cumcount().astype('str')

# Add Features
- Add the same features as in port prediction/stay duration
- Uses pivots for more features

In [ ]:
def get_pivots(port_only=False):
    #add iso3 codes to hist_visits
    ports=pd.read_parquet('ports_prep.parquet.gzip').set_index('port_index')
    hist_visits = pd.read_parquet('hist_visits.parquet.gzip')
    hist_visits = pd.merge(hist_visits, ports[['iso3']].reset_index(), how='left', on='port_index')
    hist_visits = pd.merge(hist_visits, ports[['iso3']].rename(columns={'iso3':'target_iso3'}), 
                               how='left', left_on='target_port_index', right_index=True)
    
    hist_visits['hist_connections'] = 1
    
    #Connections between ports
    port_pt = pd.pivot_table(hist_visits, index=['port_index', 'target_port_index'], 
                        values='hist_connections', aggfunc='count').fillna(0).reset_index()
    port_pt.rename(columns={'hist_connections': 'port_hist_connections'}, inplace=True)
    port_pt = port_pt.set_index(['port_index', 'target_port_index'])
    
    if port_only:
        return port_pt
    
    #travel time/stay duration between ports
    port_pt2 = pd.pivot_table(hist_visits, index=['port_index', 'target_port_index'], 
                        values=['target_travel_duration', 'target_stay_duration'], 
                         aggfunc=['mean', 'std', 'max', 'min']).reset_index()
    port_pt2 = port_pt2.set_index(['port_index', 'target_port_index'])
    port_pt2.columns = ['ports_' + '_'.join((str(j), str(k))) for j, k in port_pt2.columns]  
    
    
    #connections between countries
    iso_pt = pd.pivot_table(hist_visits, index=['iso3', 'target_iso3'], 
                        values='hist_connections', aggfunc='count').fillna(0).reset_index()
    iso_pt.rename(columns={'hist_connections': 'iso_hist_connections'}, inplace=True)
    iso_pt = iso_pt.set_index(['iso3', 'target_iso3'])

    #travel time/stay duration between countries
    iso_pt2 = pd.pivot_table(hist_visits, index=['iso3', 'target_iso3'], 
                        values=['target_travel_duration', 'target_stay_duration'], 
                         aggfunc=['mean', 'std', 'max', 'min']).reset_index()
    iso_pt2 = iso_pt2.set_index(['iso3', 'target_iso3'])
    iso_pt2.columns = ['iso3_' + '_'.join((str(j), str(k))) for j, k in iso_pt2.columns]    
    

    #merge datasets
    port_pt = pd.merge(port_pt, port_pt2, left_index=True, right_index=True)
    iso_pt = pd.merge(iso_pt, iso_pt2, left_index=True, right_index=True)

    
    #previous port --> target port connections
    prev_pt = pd.pivot_table(hist_visits, index=['previous_port_index', 'target_port_index'], 
                        values='hist_connections', aggfunc='count').fillna(0).reset_index()
    prev_pt.rename(columns={'hist_connections': 'prev_hist_connections'}, inplace=True)
    prev_pt = prev_pt.set_index(['previous_port_index', 'target_port_index'])
    
    #prev2 port --> target port connections
    prev2_pt = pd.pivot_table(hist_visits, index=['prev2_port_index', 'target_port_index'], 
                            values='hist_connections', aggfunc='count').fillna(0).reset_index()
    prev2_pt.rename(columns={'hist_connections': 'prev2_hist_connections'}, inplace=True)
    prev2_pt = prev2_pt.set_index(['prev2_port_index', 'target_port_index'])
    
    return port_pt, iso_pt, prev_pt, prev2_pt

In [ ]:
def add_features(dataset):
    port_pt, iso_pt, prev_pt, prev2_pt = get_pivots()
    ports = pd.read_parquet('ports_prep.parquet.gzip').set_index('port_index')
    distances = pd.read_excel('CERDI.xlsx').set_index(['iso1', 'iso2'])[['seadistance']]
    og_port_columns = ports.columns
    
    #add target port features
    ports.columns = ['target_' + column for column in og_port_columns]
    dataset = pd.merge(dataset, ports, how='left', left_on='target_port_index', right_index=True)
    ports.columns = og_port_columns
    
    #add port_pt, iso_pt features
    dataset = pd.merge(dataset, port_pt, how='left', left_on=['port_index', 'target_port_index'], right_index=True)
    dataset = pd.merge(dataset, iso_pt, how='left', left_on=['iso3', 'target_iso3'], right_index=True)
    dataset['port_hist_connections'] = dataset['port_hist_connections'].fillna(0)
    dataset['iso_hist_connections'] = dataset['iso_hist_connections'].fillna(0)
    
    #add fav port connections
    port_pt = port_pt.reset_index().groupby('port_index').head(1)
    dataset = pd.merge(dataset, port_pt[['port_index', 'target_port_index', 
                       'port_hist_connections']].rename(columns={'target_port_index': 'port_fav_port',
                       'port_hist_connections': 'port_fav_port_count'}), 
                       how='left', on='port_index')
    dataset['port_fav_port_count'] = dataset['port_fav_port_count'].fillna(0)
    dataset['port_fav_port'] = dataset['port_fav_port'].astype('object').fillna('Unknown').astype('category')

    ports.columns = ['port_fav_' + column for column in og_port_columns]
    dataset = pd.merge(dataset, ports[['port_fav_port_lat', 'port_fav_port_long']], 
                       how='left', left_on='target_port_index', right_index=True)
    ports.columns = og_port_columns
    
    #add previous and prev2 pt features
    dataset = pd.merge(dataset, prev_pt, how='left', left_on=['previous_port_index', 'target_port_index'], right_index=True)
    dataset = pd.merge(dataset, prev2_pt, how='left', left_on=['prev2_port_index', 'target_port_index'], right_index=True)
    dataset['prev_hist_connections'] = dataset['prev_hist_connections'].fillna(0)
    dataset['prev2_hist_connections'] = dataset['prev2_hist_connections'].fillna(0)    
    
    #add seadistance curr -- target
    dataset = pd.merge(dataset, distances[['seadistance']], how='left',
                       left_on=['iso3', 'target_iso3'], right_index=True)
    dataset['seadistance'] = dataset['seadistance'].fillna(0)

    #add seadistance prev -- target
    dataset = pd.merge(dataset, distances[['seadistance']].rename(columns={'seadistance': 'prev_tar_seadistance'}), 
                       how='left', left_on=['previous_iso3', 'target_iso3'], right_index=True)
    dataset['prev_tar_seadistance'] = dataset['prev_tar_seadistance'].fillna(0)

    #add seadistance prev2 -- target
    dataset = pd.merge(dataset, distances[['seadistance']].rename(columns={'seadistance': 'prev2_tar_seadistance'}), 
                       how='left', left_on=['prev2_iso3', 'target_iso3'], right_index=True)
    dataset['prev2_tar_seadistance'] = dataset['prev2_tar_seadistance'].fillna(0)
    
    #add euclidean distances
    dataset['eucl_cur_tar'] = np.sqrt((dataset['port_lat'] - dataset['target_port_lat'])**2 + 
                                 (dataset['port_long'] - dataset['target_port_long'])**2)
    dataset['eucl_cur_prev'] = np.sqrt((dataset['port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['port_long'] - dataset['previous_port_long'])**2)
    dataset['eucl_tar_prev'] = np.sqrt((dataset['target_port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['target_port_long'] - dataset['previous_port_long'])**2)
    dataset['eucl_ves_tar'] = np.sqrt((dataset['mean_port_lat'] - dataset['target_port_lat'])**2 + 
                                 (dataset['mean_port_long'] - dataset['target_port_long'])**2)
    dataset['eucl_ves_cur'] = np.sqrt((dataset['mean_port_lat'] - dataset['port_lat'])**2 + 
                                 (dataset['mean_port_long'] - dataset['port_long'])**2)
    dataset['eucl_ves_prev'] = np.sqrt((dataset['mean_port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['mean_port_long'] - dataset['previous_port_long'])**2)
    
    dataset['eucl_cur_prev2'] = np.sqrt((dataset['port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['port_long'] - dataset['prev2_port_long'])**2)
    dataset['eucl_tar_prev2'] = np.sqrt((dataset['target_port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['target_port_long'] - dataset['prev2_port_long'])**2)
    dataset['eucl_prev_prev2'] = np.sqrt((dataset['previous_port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['previous_port_long'] - dataset['prev2_port_long'])**2)
    dataset['eucl_ves_prev2'] = np.sqrt((dataset['mean_port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['mean_port_long'] - dataset['prev2_port_long'])**2)    

    dataset['eucl_ves_fav_cur'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['port_long'])**2)
    dataset['eucl_ves_fav_tar'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['target_port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['target_port_long'])**2)
    dataset['eucl_ves_fav_prev'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['previous_port_long'])**2)
    dataset['eucl_ves_fav_prev2'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['prev2_port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['prev2_port_long'])**2) 
    dataset['eucl_ves_fav_ves'] = np.sqrt((dataset['vessel_fav_port_lat'] - dataset['mean_port_lat'])**2 + 
                                 (dataset['vessel_fav_port_long'] - dataset['mean_port_long'])**2)
    
    dataset['eucl_port_fav_tar'] = np.sqrt((dataset['port_fav_port_lat'] - dataset['target_port_lat'])**2 + 
                                 (dataset['port_fav_port_long'] - dataset['target_port_long'])**2)
    dataset['eucl_port_fav_prev'] = np.sqrt((dataset['port_fav_port_lat'] - dataset['previous_port_lat'])**2 + 
                                 (dataset['port_fav_port_long'] - dataset['previous_port_long'])**2)
    dataset['eucl_port_fav_ves'] = np.sqrt((dataset['port_fav_port_lat'] - dataset['mean_port_lat'])**2 + 
                                 (dataset['port_fav_port_long'] - dataset['mean_port_long'])**2)    
    dataset['eucl_port_fav_ves_fav'] = np.sqrt((dataset['port_fav_port_lat'] - dataset['vessel_fav_port_lat'])**2 + 
                                 (dataset['port_fav_port_long'] - dataset['vessel_fav_port_long'])**2)   
    
    #add rankings
    dataset['rank_port_connections'] = dataset.groupby('identifier')['port_hist_connections'].rank(axis=0, 
                                                                                        method='min', ascending=False)
    dataset['rank_iso_connections'] = dataset.groupby('identifier')['iso_hist_connections'].rank(axis=0, 
                                                                                        method='min', ascending=False)
    dataset['rank_prev_connections'] = dataset.groupby('identifier')['prev_hist_connections'].rank(axis=0, 
                                                                                        method='min', ascending=False)
    dataset['rank_prev2_connections'] = dataset.groupby('identifier')['prev2_hist_connections'].rank(axis=0, 
                                                                                        method='min', ascending=False)
    dataset['rank_seadistance'] = dataset.groupby('identifier')['seadistance'].rank(axis=0, method='min')
    dataset['rank_prev_tar_seadistance'] = dataset.groupby('identifier')['prev_tar_seadistance'].rank(axis=0, method='min')
    dataset['rank_eucl_cur_tar'] = dataset.groupby('identifier')['eucl_cur_tar'].rank(axis=0, method='min')
    dataset['rank_eucl_tar_prev'] = dataset.groupby('identifier')['eucl_tar_prev'].rank(axis=0, method='min')
    dataset['rank_eucl_tar_ves_fav'] = dataset.groupby('identifier')['eucl_ves_fav_tar'].rank(axis=0, method='min')
    dataset['rank_eucl_tar_mean'] = dataset.groupby('identifier')['eucl_ves_tar'].rank(axis=0, method='min')
    dataset['rank_eucl_port_fav_tar'] = dataset.groupby('identifier')['eucl_port_fav_tar'].rank(axis=0, method='min')
    
    #rename a column
    dataset.rename(columns={'n_visits_y': 'n_visits'}, inplace=True)
    
    #add some difference metrics
    for col in ['port_lat', 'port_long', 'n_visits', 'n_unique_vessels',
           'n_high_speed', 'n_medium_speed', 'n_Chemical/Oil Tanker', 'n_Container Ship', 'n_Crude Oil Tanker',
           'n_General Cargo Ship', 'n_Tanker', 'n_large_length', 'n_medium_length',
           'n_small_length', 'n_very large_length', 'n_large_depth',
           'n_medium_depth', 'n_small_depth', 'n_very large_depth', 'port_avg_distance_to_port', 'port_avg_travel_duration',
           'port_avg_stay_duration' ]:
        dataset[f'prev_curr_diff_{col}'] = abs(dataset[col] - dataset[f'previous_{col}'])
        dataset[f'curr_targ_diff_{col}'] = abs(dataset[f'target_{col}'] - dataset[col])
        dataset[f'fav_targ_diff_{col}'] = abs(dataset[f'target_{col}'] - dataset[f'vessel_fav_{col}'])   
    
    
    dataset['prev_speed_coor'] = (dataset['eucl_cur_prev']*111) / np.exp(dataset['previous_travel_duration'])
    dataset['prev_speed_sead'] = dataset['previous_seadistance'] / np.exp(dataset['previous_travel_duration'])
    dataset['prev2_speed_coor'] = (dataset['eucl_prev_prev2']*111) / np.exp(dataset['prev2_travel_duration'])
    dataset['prev2_speed_sead'] = dataset['prev2_prev_seadistance'] / np.exp(dataset['prev2_travel_duration'])
    
    dataset['exp_trav_prev_coor'] = np.log((dataset['eucl_cur_tar']*111) / dataset['prev_speed_coor'])
    dataset['exp_trav_prev_sead'] = np.log(dataset['seadistance'] / dataset['prev_speed_sead'])
    dataset['exp_trav_prev2_coor'] = np.log((dataset['eucl_cur_tar']*111) / dataset['prev2_speed_coor'])
    dataset['exp_trav_prev2_sead'] = np.log(dataset['seadistance'] / dataset['prev2_speed_sead'])
        
    return dataset

In [ ]:
dataset = add_features(df)
print(dataset.shape)
dataset.head()

### Mapping
- Use the same mapping as the one from port predictions

In [ ]:
mappings = pickle.load(open('mappings.p', 'rb'))

for column in dataset.columns[dataset.dtypes == 'object']:
    if (column != 'identifier') and ('iso3' not in column) and ('portname' not in column):
        print(column)
        dataset[column] = dataset[column].map(mappings[column]).astype('category')

# TRAIN TEST SPLIT
- Split the data on time
- There are some columns we do not want to use in the model. For example using the mmsi prohibits the model to be effective for unseen vessels. Also the features with many categories (such as mmsi) could induce overfitting

In [ ]:
dataset = dataset.set_index(['identifier', 'port_index', 'target_port_index'])

In [ ]:
dataset.shape

In [ ]:
dataset.index.levels[0].nunique()

In [ ]:
train_cutoff = int(len(dataset)*0.6)
val_cutoff = int(len(dataset)*0.8)

traindf = dataset.iloc[:train_cutoff]
valdf = dataset.iloc[train_cutoff:val_cutoff]
testdf = dataset.iloc[val_cutoff:]

In [ ]:
cols_not_train = ['portname', 'target_portname', 'prev2_portname',
                 'mmsi_iso3', 'iso3', 'previous_iso3', 'prev2_iso3', 'target_iso3',
                 'prev2_port_index', 'vessel_fav_portname', 'vessel_fav_iso3', 'vessel_fav_port_index',
                 'port_fav_port', 'target_travel_duration']

cols_train = traindf.drop(cols_not_train, 1).columns

# FEATURE SELECTION
- Define a model
- Fit a model
- Select the features

In [ ]:
model = LGBMRegressor(max_depth=10, n_estimators=500, learning_rate=.05, verbose=1,
                   min_child_samples=5000, num_leaves=25, metric=['rmse', 'mape'])

In [ ]:
model.fit(traindf[cols_train], traindf['target_travel_duration'].values, 
         eval_set=[(traindf[cols_train], traindf['target_travel_duration'].values), 
                   (valdf[cols_train], valdf['target_travel_duration'].values)],
         eval_metric=['rmse', 'mape'],
         verbose=10, early_stopping_rounds=250)

In [ ]:
pd.DataFrame({'Feature': cols_train, 'Importance': model.feature_importances_}).sort_values('Importance').tail(25)

In [ ]:
new_cols = pd.DataFrame({'Feature': cols_train, 
                         'Importance': model.feature_importances_}).sort_values('Importance').tail(50)['Feature'].values

# MODEL
- Define a model
- Train a model on the selected features
- Check performance
- Save the used features and the model
- Save the predictions to be analyzed in the results notebook

In [ ]:
model = LGBMRegressor(max_depth=15, n_estimators=5000, learning_rate=.025, verbose=1,
                   min_child_samples=500, num_leaves=50, metric=['mape', 'rmse'])

In [ ]:
model.fit(traindf[new_cols], traindf['target_travel_duration'].values, 
         eval_set=[(traindf[new_cols], traindf['target_travel_duration'].values), 
                   (valdf[new_cols], valdf['target_travel_duration'].values)],
         eval_metric=['mape', 'rmse'],
         verbose=10, early_stopping_rounds=250)

In [ ]:
import lightgbm as lgb
lgb.plot_metric(model)

In [ ]:
prediction = model.predict(testdf[new_cols])

print ('MAPE (test):', MAPE(np.exp(testdf['target_travel_duration']), np.exp(prediction)))
print ('RMSE (test):', np.sqrt(mse(np.exp(testdf['target_travel_duration']), np.exp(prediction))))
print()

print(np.corrcoef(np.exp(testdf['target_travel_duration']), np.exp(prediction)))
print()

plt.plot(np.exp(testdf['target_travel_duration']), np.exp(prediction), 'o', markersize=0.5)

In [ ]:
pd.DataFrame({'Feature': new_cols, 'Importance': model.feature_importances_}).sort_values('Importance').tail(25)

In [ ]:
pickle.dump(new_cols, open('travel_cols.p', 'wb'))
pickle.dump(model, open('travel_model.p', 'wb'))

In [ ]:
results = testdf[['port_lat', 'port_long', 'target_port_lat', 
                  'target_port_long', 'eucl_cur_tar', 'target_travel_duration']].reset_index()
results['prediction'] = np.exp(prediction)
results['target_travel_duration'] = np.exp(results['target_travel_duration'])

In [ ]:
results.to_csv('travel_prediction.csv', index=False)

In [ ]:
results.head()

# Baseline
- Baseline uses the historic visits as well as the train dataset
- Computes the average travel duration between ports
- Save the predictions to evaluate them in the Results notebook

In [ ]:
hist_visits = pd.read_parquet('hist_visits.parquet.gzip')
hist_visits = hist_visits[['port_index', 'target_port_index', 'target_travel_duration']]
hist_visits = pd.concat([hist_visits, traindf[['target_travel_duration']].reset_index()])
hist_visits = hist_visits.drop('identifier', 1)

print(hist_visits.shape)
hist_visits.head()

In [ ]:
pt = pd.pivot_table(hist_visits, index=['port_index', 'target_port_index'], 
                    values='target_travel_duration', aggfunc='mean').reset_index()

print(pt.shape)
pt.head()

In [ ]:
testbase = pd.merge(testdf[['port_lat', 'port_long', 'target_port_lat', 
                  'target_port_long', 'eucl_cur_tar', 'target_travel_duration']].reset_index(), 
                    pt[['port_index', 'target_port_index', 
                        'target_travel_duration']].rename(columns={'target_travel_duration':'prediction'}),
                    how='left', on=['port_index', 'target_port_index'])

testbase['prediction'] = testbase['prediction'].fillna(testbase['prediction'].mean())

print(testbase.shape)
testbase.head()

In [ ]:
print ('MAPE (test):', MAPE(np.exp(testbase['target_travel_duration']), np.exp(testbase['prediction'])))
print ('RMSE (test):', np.sqrt(mse(np.exp(testbase['target_travel_duration']), np.exp(testbase['prediction']))))
print()

print(np.corrcoef(np.exp(testbase['target_travel_duration']), np.exp(testbase['prediction'])))
print()

plt.plot(np.exp(testbase['target_travel_duration']), np.exp(testbase['prediction']), 'o', markersize=0.5)

In [ ]:
testbase.to_csv('BASELINE_travel_prediction.csv', index=False)